In [1]:
import datetime

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras import backend as K
K.set_image_data_format('channels_last')

Using TensorFlow backend.


1 - Train a simple convnet on the MNIST dataset the first 5 digits [0..4].

2 - Freeze convolutional layers and fine-tune dense layers
   for the classification of digits [5..9].

In [2]:
now = datetime.datetime.now

batch_size = 128
num_classes = 5
epochs = 5

In [3]:
# input image dimensions
img_rows, img_cols = 28, 28
# number of convolutional filters to use
filters = 32
# size of pooling area for max pooling
pool_size = 2
# convolution kernel size
kernel_size = 3

In [4]:
input_shape = (img_rows, img_cols, 1)

### 载入数据集

In [5]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [6]:
# create two datasets one with digits below 5 and one with 5 and above
x_train_lt5 = x_train[y_train < 5]
y_train_lt5 = y_train[y_train < 5]
x_test_lt5 = x_test[y_test < 5]
y_test_lt5 = y_test[y_test < 5]

x_train_gte5 = x_train[y_train >= 5]
y_train_gte5 = y_train[y_train >= 5] - 5
x_test_gte5 = x_test[y_test >= 5]
y_test_gte5 = y_test[y_test >= 5] - 5

### 建模

In [7]:
def train_model(model, train, test, num_classes):
    x_train = train[0].reshape((train[0].shape[0],) + input_shape)
    x_test = test[0].reshape((test[0].shape[0],) + input_shape)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')

    # convert class vectors to binary class matrices
    y_train = to_categorical(train[1], num_classes)
    y_test = to_categorical(test[1], num_classes)

    model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])

    t = now()
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test))
    print('Training time: %s' % (now() - t))
    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])

In [8]:
# define two groups of layers: feature (convolutions) and classification (dense)
feature_layers = [
    Conv2D(filters, kernel_size,
           padding='valid',
           input_shape=input_shape),
    Activation('relu'),
    Conv2D(filters, kernel_size),
    Activation('relu'),
    MaxPooling2D(pool_size=pool_size),
    Dropout(0.25),
    Flatten(),
]

classification_layers = [
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(num_classes),
    Activation('softmax')
]

In [9]:
# create complete model
model = Sequential(feature_layers + classification_layers)

### 训练模型（前5个数字 0,1,2,3,4）

In [10]:
# train model for 5-digit classification [0..4]
train_model(model,
            (x_train_lt5, y_train_lt5),
            (x_test_lt5, y_test_lt5), num_classes)

x_train shape: (30596, 28, 28, 1)
30596 train samples
5139 test samples
Train on 30596 samples, validate on 5139 samples
Epoch 1/5
30596/30596 [==============================] - 45s 1ms/step - loss: 0.1640 - acc: 0.9476 - val_loss: 0.0334 - val_acc: 0.9889
Epoch 2/5
30596/30596 [==============================] - 44s 1ms/step - loss: 0.0470 - acc: 0.9854 - val_loss: 0.0166 - val_acc: 0.9940
Epoch 3/5
30596/30596 [==============================] - 43s 1ms/step - loss: 0.0323 - acc: 0.9904 - val_loss: 0.0095 - val_acc: 0.9969
Epoch 4/5
30596/30596 [==============================] - 43s 1ms/step - loss: 0.0255 - acc: 0.9922 - val_loss: 0.0139 - val_acc: 0.9949
Epoch 5/5
30596/30596 [==============================] - 43s 1ms/step - loss: 0.0214 - acc: 0.9933 - val_loss: 0.0084 - val_acc: 0.9971
Training time: 0:03:38.694125
Test score: 0.008439281038627518
Test accuracy: 0.997081144191477


### 迁移学习（后5个数字 5,6,7,8,9）

In [11]:
# freeze feature layers and rebuild model
for l in feature_layers:
    l.trainable = False

In [12]:
# transfer: train dense layers for new classification task [5..9]
train_model(model,
            (x_train_gte5, y_train_gte5),
            (x_test_gte5, y_test_gte5), num_classes)

x_train shape: (29404, 28, 28, 1)
29404 train samples
4861 test samples
Train on 29404 samples, validate on 4861 samples
Epoch 1/5
29404/29404 [==============================] - 18s 624us/step - loss: 0.2422 - acc: 0.9311 - val_loss: 0.0485 - val_acc: 0.9854
Epoch 2/5
29404/29404 [==============================] - 18s 619us/step - loss: 0.0786 - acc: 0.9764 - val_loss: 0.0352 - val_acc: 0.9877
Epoch 3/5
29404/29404 [==============================] - 18s 617us/step - loss: 0.0600 - acc: 0.9813 - val_loss: 0.0291 - val_acc: 0.9903
Epoch 4/5
29404/29404 [==============================] - 18s 620us/step - loss: 0.0483 - acc: 0.9853 - val_loss: 0.0242 - val_acc: 0.9920
Epoch 5/5
29404/29404 [==============================] - 18s 616us/step - loss: 0.0439 - acc: 0.9872 - val_loss: 0.0255 - val_acc: 0.9907
Training time: 0:01:31.265919
Test score: 0.02550407264179902
Test accuracy: 0.9907426454358277
